In [14]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

In [15]:
import utils
from genotypes import Genotype

In [16]:
# tf.enable_eager_execution()

In [17]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.025,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "batch_size": 4,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [4, 4],
    "save_checkpoints_steps": 100,
    "model_dir": './outputdir'
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [18]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn():
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = 16, 16

        # Create a dictionary describing the features.  
        image_feature_description = {
            'name': tf.FixedLenFeature([], tf.string),  
            'label_encoded': tf.FixedLenFeature([], tf.string),
            'encoded': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            image= feature['encoded']
            label = feature['label_encoded']
            name = feature['name']

            image = tf.image.decode_png(image, channels=3)
            label = tf.image.decode_png(label, channels=3)

            image = tf.cast(image, tf.float32)
            image = tf.image.resize(image, (W, H))
            label = tf.cast(label, tf.float32)
            label = tf.image.resize(label, (W, H))

            return image, label

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)

        return dataset
    
    return _input_fn

In [19]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn():
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
            x_train = np.random.randint(0, 256, (NUM_IMAGES // batch_size, batch_size, W, H, 3)).astype(np.float32)
            y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES // batch_size, batch_size, W, H, 1)).astype(np.float32)
            x_valid = np.random.randint(0, 256, (NUM_IMAGES // batch_size, batch_size, W, H, 3)).astype(np.float32)
            y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES // batch_size, batch_size, W, H, 1)).astype(np.float32)
            
            ds = (x_train, x_valid), (y_train, y_valid)
            dataset = tf.data.Dataset.from_tensor_slices(ds)
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
            y_train = np.random.randint(0, 6, (NUM_IMAGES, W, H, 1)).astype(np.float32)
            
            ds = (x_train, y_train)
            dataset = tf.data.Dataset.from_tensor_slices(ds)
            num_epochs = 1 # end-of-input after this
#         w, h, c = dataset.shape
        dataset = dataset.repeat(num_epochs)
        return dataset
    
    return _input_fn

### Model function

In [20]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, genotype):
        self.genotype = genotype
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
    
    def end(self, session):
        normal_gene_op = self.genotype.normal
        reduce_gene_op = self.genotype.reduce
        
        self.global_step = session.run(self.global_step)
        normal_gene = session.run(normal_gene_op)
        reduce_gene = session.run(reduce_gene_op)
        
        genotype = Genotype(
            normal=normal_gene, normal_concat=self.genotype.normal_concat,
            reduce=reduce_gene, reduce_concat=self.genotype.reduce_concat
        )
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [21]:
def model_fn(features, labels, mode):
    criterion = tf.losses.sigmoid_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion)
    print("$$$$$$$$$$$$$$$$$", tf.trainable_variables())
    global_step = tf.train.get_global_step()
    learning_rate_min = tf.constant(args.learning_rate_min)
    
    learning_rate = tf.train.exponential_decay(
        args.learning_rate,
        global_step,
        decay_rate=args.learning_rate_decay,
        decay_steps=args.num_batches_per_epoch,
        staircase=True,
    )
    
    lr = tf.maximum(learning_rate, learning_rate_min)
    tf.summary.scalar('learning_rate', lr)
    
    optimizer = tf.train.MomentumOptimizer(lr, args.momentum)
    criterion = tf.losses.sigmoid_cross_entropy
    
    eval_hooks = None
    
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
        architect = Architect(model, args)
        # architect step
        architect_step = architect.step(input_train=x_train,
                                        target_train=y_train,
                                        input_valid=x_valid,
                                        target_valid=y_valid,
                                        unrolled=args.unrolled,
                                        )


        w_var = model.get_thetas()
        loss = model._loss(preds, y_train)
        grads = tf.gradients(loss, w_var)
        clipped_gradients, norm = tf.clip_by_global_norm(grads, args.grad_clip)
        train_op = optimizer.apply_gradients(zip(clipped_gradients, w_var), global_step=tf.train.get_global_step())

        miou = tf.metrics.mean_iou(
                    labels=y_train,
                    predictions=preds,
                    num_classes=args.num_classes
                )
        acc = tf.metrics.accuracy(labels=y_train, 
                                  predictions=preds)
        eval_metric_ops = {
            "miou": miou,
            "accuracy": acc
        }
    
    elif mode == tf.estimator.ModeKeys.EVAL:
#         global_step = tf.train.get_global_step()
        genotype = model.genotype()
        gene_saver = GeneSaver(genotype)
        eval_hooks = [gene_saver]
        
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        preds = model(x_valid)
        loss = model._loss(preds, y_valid)
        miou = tf.metrics.mean_iou(
                    labels=y_valid,
                    predictions=preds,
                    num_classes=args.num_classes
                )
        acc = tf.metrics.accuracy(labels=y_valid, predictions=preds)
        eval_metric_ops = {
            "miou": miou,
            "accuracy": acc
        }
        
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metric_ops = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks
    )

### Estimator

In [22]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn2(filename = '../../datasets/infer/infer-00000-00007.tfrecords',
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.batch_size)

In [23]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


In [24]:
config = tf.estimator.RunConfig(save_checkpoints_steps=args.save_checkpoints_steps,
                                model_dir=args.model_dir)

In [25]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(output_dir):
    estimator = tf.estimator.Estimator(model_fn = model_fn, 
                         config=config)
    train_spec = tf.estimator.TrainSpec(input_fn = get_train(),
                                    max_steps = 1000)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = get_train(),
                                  steps = None, throttle_secs=600)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [27]:
estimator = tf.estimator.Estimator(model_fn = model_fn, 
                     config=config)
estimator.train(input_fn = get_train(), steps = 1)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb49aa5eb90>, '_model_dir': './outputdir', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically 

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'network_1_1/sequential_1/conv2d_14/kernel:0' shape=(3, 3, 3, 9) dtype=float32>", "<tf.Variable 'network_1_1/sequential_1/batch_normalization_1/gamma:0' shape=(9,) dtype=float32>", "<tf.Variable 'network_1_1/sequential_1/batch_normalization_1/beta:0' shape=(9,) dtype=float32>", "<tf.Variable 'conv2d_11/kernel:0' shape=(1, 1, 9, 3) dtype=float32>", "<tf.Variable 'conv2d_12/kernel:0' shape=(1, 1, 9, 3) dtype=float32>", "<tf.Variable 'conv2d_13/kernel:0' shape=(1, 1, 9, 6) dtype=float32>", "<tf.Variable 'conv2d_14/kernel:0' shape=(1, 1, 12, 6) dtype=float32>", "<tf.Variable 'conv2d_15/kernel:0' shape=(1, 1, 12, 3) dtype=float32>", "<tf.Variable 'conv2d_16/kernel:0' shape=(1, 1, 12, 3) dtype=float32>", "<tf.Variable 'conv2d_17/kernel:0' shape=(1, 1, 24, 6) dtype=float32>", "<tf.Variable 'conv2d_18/kernel:0' shape=(1, 1, 24, 3) dtype=float32>", "<tf.Va

In [11]:
train_and_evaluate(args.model_dir)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbe35be9250>, '_model_dir': './outputdir', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation l

## Extra Test

In [5]:
criterion = tf.losses.sigmoid_cross_entropy
model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)

In [6]:
(x_train, x_valid), (y_train, y_valid) = get_train()().make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [7]:
logits = model(x_train)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'IteratorGetNext:2' shape=(4, 4, 4, 1) dtype=float32>

In [10]:
miou = tf.metrics.mean_iou(
    labels=y_train,
    predictions=logits,
    num_classes=args.num_classes
)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [11]:
global_init_op = tf.global_variables_initializer()
local_init_op = tf.local_variables_initializer()

In [14]:
with tf.Session() as sess:
    sess.run([global_init_op, local_init_op])
    pred = sess.run(logits)
    miou_out = sess.run(miou)

In [15]:
pred

array([[[[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]]],


       [[[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]]],


       [[[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]]],


       [[[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]],

        [[3.],
         [3.],
         [3.],
         [3.]]]], d

In [16]:
tf.Session().run(y_train)

array([[[[3.],
         [3.],
         [0.],
         [4.]],

        [[5.],
         [1.],
         [3.],
         [1.]],

        [[0.],
         [1.],
         [1.],
         [4.]],

        [[0.],
         [1.],
         [0.],
         [1.]]],


       [[[0.],
         [3.],
         [2.],
         [0.]],

        [[1.],
         [0.],
         [2.],
         [2.]],

        [[0.],
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [4.],
         [4.]]],


       [[[2.],
         [2.],
         [5.],
         [5.]],

        [[4.],
         [0.],
         [0.],
         [5.]],

        [[1.],
         [2.],
         [4.],
         [0.]],

        [[4.],
         [5.],
         [2.],
         [1.]]],


       [[[2.],
         [1.],
         [2.],
         [5.]],

        [[2.],
         [3.],
         [4.],
         [0.]],

        [[1.],
         [0.],
         [3.],
         [0.]],

        [[5.],
         [2.],
         [2.],
         [3.]]]], d

In [17]:
miou_out

(0.0, array([[ 0.,  0.,  0., 11.,  0.,  0.],
        [ 0.,  0.,  0., 14.,  0.,  0.],
        [ 0.,  0.,  0.,  8.,  0.,  0.],
        [ 0.,  0.,  0., 12.,  0.,  0.],
        [ 0.,  0.,  0., 11.,  0.,  0.],
        [ 0.,  0.,  0.,  8.,  0.,  0.]]))